# Proxy

## O que é?

O padrão _Proxy_ sugere a criação de um objeto com o qual o cliente interagirá e que controla o acesso atributos de outros objetos.

## Por quê?

Esse padrão nos dá controle sobre todas as interações entre o cliente e o objeto real, permitindo que otimizemos uso de memória, controlemos o acesso de atributos de acordo com o usuário, abstraiamos interações mais complexas.

O livro cita 4 tipos clássicos de _proxies_.

- _Virtual Proxy_: Um objeto que apresenta um comportamento idêntico a um objeto real, porém que realiza operação custosas apenas quando realmente necessário (e.g.: Spark DFs).
- _Remote Proxy_: Atua como um objeto local, abstraindo as operações necessárias para acessar um objeto remoto (e.g.: VMs).
- Protection Proxy_: Permite ou nega o acesso de usuários a certo atributos do objeto real.
- _Smart Reference_: Usado para controlar a criação e destruição de objetos. Geralmente essas estruturas mantém um contador de referências que ao chegar a zero deleta o objeto real.

## Estrutura

![struct](assets/proxy-struct.jpg)

## Exemplo 1

Imaginemos que uma empresa tem um sistem de ORM. Esse sistema performa operações basicas como criar, destruir e atualizar o _schema_. Como esperado, existem multiplas posições na empresa que precisam interagir com esse database, como analistas, engenheiros e engenheiros sr.

In [89]:
class Table:
    def __init__(self, name, schema=None):
        self.name = name
        self.schema = schema or {}
        
    def __repr__(self):
        return "Table(name={name}, schema=({schema}))".format(
            name=self.name,
            schema=", ".join(f"{field}: {repr(type_)}" for field, type_ in self.schema.items())
        )

class DB:
    def __init__(self, name: str):
        self.name = name
        self._tables = None
    
    @property
    def tables(self):
        if self._tables is None:
            global tables
            self._tables = tables
        return self._tables
    
    def drop_table(self, name):
        del self._tables[name]
        
    def add_table(self, name, schema):
        if name in self.tables:
            raise Exception("table already exists")
        self._tables[name] = Table(name, schema)
        
    def __repr__(self):
        return f"Database({self.name})"

In [38]:
tables = {}

db = DB("unsafe")
db

Database(unsafe)

In [39]:
db.tables

{}

In [40]:
db.add_table("users", {"id": str, "name": str, "age": int})
db.tables

{'users': Table(name=users, schema=(id: <class 'str'>, name: <class 'str'>, age: <class 'int'>))}

In [41]:
db.drop_table("users")
db.tables

{}

Para evitar erros, é necessário que um sistema de controle de acesso seja criado. Uma forma fazê-lo é implementá-lo diretamente no código do ORM.

In [57]:
import os


role_levels = {
    "analyst": 10,
    "engineer": 30,
    "sr-engineer": 50,
}


def get_access_level():
    """
    Return the access level for the current user.
    """
    role = os.environ.get("DB_ROLE")
    return role_levels.get(role, 0)


def restrict_access(min_access_level):
    """
    Restrict access to functions.
    """
    def decorator(func):
        def wrapper(*args, **kwargs):
            if get_access_level() < min_access_level:
                raise Exception("Unauthorised.")
            return func(*args, **kwargs)
        return wrapper
    return decorator

In [90]:
class DB:
    def __init__(self, name: str):
        self.name = name
        self._tables = None
    
    @property
    def tables(self):
        if self._tables is None:
            global tables
            self._tables = tables
        return self._tables
    
    @restrict_access(50)
    def drop_table(self, name):
        del self._tables[name]
        
    @restrict_access(30)
    def add_table(self, name, schema):
        if name in self.tables:
            raise Exception("table already exists")
        self._tables[name] = Table(name, schema)
        
    def __repr__(self):
        return f"Database({self.name})"

Agora, se tentarmos repetir as operações...

In [59]:
os.environ["DB_ROLE"] = "engineer"

In [60]:
tables = {}

db = DB("unsafe")
db

Database(unsafe)

In [61]:
db.tables

{}

In [62]:
db.add_table("users", {"id": str, "name": str, "age": int})
db.tables

{'users': Table(name=users, schema=(id: <class 'str'>, name: <class 'str'>, age: <class 'int'>))}

In [63]:
db.drop_table("users")
db.tables

Exception: Unauthorised.

O problem aqui é que estamos misturando com a lógica de aplicação. Talvez diferentes DBs tenham padrões de acesso diferentes.

Uma forma de remediar isso é usando um _Protection Proxy_.

In [99]:
import abc

class DBBase(abc.ABC):
    
#     nós deveríamos adicionar `tables` como uma `abstractmethod`
#     porém por motivos que serão discutidos mais em a frente não faremos isso
#     @property
#     def tables(self):
#         pass
    
    @abc.abstractmethod
    def drop_table(self, name):
        pass
        
    @abc.abstractmethod
    def add_table(self, name, schema):
        pass
    
    def __repr__(self):
        return f"Database({self.name})"    


class RealDB(DBBase):
    def __init__(self, name: str):
        self.name = name
        self._tables = None
    
    @property
    def tables(self):
        if self._tables is None:
            global tables
            self._tables = tables
        return self._tables
    
    def drop_table(self, name):
        del self._tables[name]
        
    def add_table(self, name, schema):
        if name in self.tables:
            raise Exception("table already exists")
        self._tables[name] = Table(name, schema)
        
class ProxyDB(DBBase):
    def __init__(self, name: str):
        self.name = name
        self._db = dbs[name]
    
    @property
    def tables(self):
        return self._db._tables
    
    @restrict_access(50)
    def drop_table(self, name):
        return self._db.drop_table(name)
        
    @restrict_access(30)
    def add_table(self, name, schema):
        return self._db.add_table(name, schema)

In [100]:
os.environ["DB_ROLE"] = "engineer"

In [101]:
dbs = {"unsafe": RealDB("unsafe")}
tables = {}

Um dos problemas desse padrão é "Como identificar um objeto que ainda não foi instanciado?". Nesse caso, nós temos que criar um identificador universal para o banco de dados. 

In [102]:
db = ProxyDB("unsafe")
db

Database(unsafe)

In [103]:
db.tables

In [104]:
db.add_table("users", {"id": str, "name": str, "age": int})
db.tables

{'users': Table(name=users, schema=(id: <class 'str'>, name: <class 'str'>, age: <class 'int'>))}

In [105]:
db.drop_table("users")
db.tables

Exception: Unauthorised.

Um dos problemas aqui é o quão custosa é a manutenção dessa interface. Em algumas linguagens como Python, nós podemos mitigar esse problema sobreescrevendo a operação de acessar atributos das classes.

In [121]:
class ProxyDB(DBBase):
    def __init__(self, name: str):
        self.name = name
        self._db = dbs[name]
        
    def __getattr__(self, name):
        return vars(self).get(name, None) or getattr(self._db, name)
    
    @restrict_access(50)
    def drop_table(self, name):
        return self._db.drop_table(name)
        
    @restrict_access(30)
    def add_table(self, name, schema):
        return self._db.add_table(name, schema)

In [124]:
dbs = {"unsafe": RealDB("unsafe")}
tables = {}
db = ProxyDB("unsafe")
db

Database(unsafe)

In [125]:
db.tables

{}

In [126]:
db.add_table("users", {"id": str, "name": str, "age": int})
db.tables

{'users': Table(name=users, schema=(id: <class 'str'>, name: <class 'str'>, age: <class 'int'>))}

In [127]:
db.drop_table("users")
db.tables

Exception: Unauthorised.

Por outro lado, eu tive que abrir mão de definir `tables` como um método abstrato.

## Prós e contras:

### Pros
- Permite maior robustez na criacao de objetos complexos
- Cria uma separacao clara entre a construcao do objeto e o objeto em si
- Evita construtores ou factory methods com muitos parâmetros

### Cons
- Aumenta a complexidade do sistema como um todo
- Duplicacao de código (propriedades, getters, etc) entre Builder e Product
- Exige que o cliente conheca a implementacao do objeto

## Discussao:

Like the Abstract Factory pattern, the Builder pattern requires that you define an interface, which will be used by clients to create complex objects in pieces. In the MazeBuilder example, there are BuildMaze(), BuildRoom() and BuildDoor() methods, along with a GetMaze() method. How does the Builder pattern allow one to add new methods to the Builder's interface, without having to change each and every sub-class of the Builder?